In [27]:
import requests
from bs4 import BeautifulSoup
from deep_translator import (GoogleTranslator,MicrosoftTranslator,MyMemoryTranslator)
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline

def get_text(url):
    rs = requests.get(url)
    root = BeautifulSoup(rs.content, 'html.parser')
    article = root.select_one('article')
    return article


def get_mass_result(url):
    text = get_text(url)
    mass = [i if len(i) > 15 else 0 for i in text.text.replace('  ','').split('\n')]

    while mass.count(0) != 0:
        mass.remove(0)
    print(title)
    #print(GoogleTranslator(source='russian', target='ukrainian').translate(mass[0]))
    print('\n\n')
    text = ''
    for i in mass[1:]:
        text+=i
        text+=' '
    text = [i if len(i)>5 else None for i in text.split('. ')]
    while text.count(None) != 0:
        text.remove(None)

    english_text_mass = [GoogleTranslator(source='russian', target='english').translate(i) for i in text]
    english_text = ''
    for i in english_text_mass:
        english_text+=i
        english_text+=' '
    return english_text


def summary1(english_text):
    tokenizer = AutoTokenizer.from_pretrained('t5-base')
    model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)
    inputs = tokenizer.encode("summarize: " + english_text,return_tensors='pt',max_length=512,truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=5., num_beams=2)
    summary = tokenizer.decode(summary_ids[0])
    return (GoogleTranslator(source='english', target='ukrainian').translate(summary))


def summary2(english_text):
    summarization = pipeline("summarization")
    summary_text = summarization(english_text)[0]['summary_text']
    return (GoogleTranslator(source='english', target='ukrainian').translate(summary_text))


In [28]:
url = 'https://itc.ua/news/youtube-testiruet-funkcziyu-avtomaticheskogo-opredeleniya-produktov-v-video/'
rs = requests.get(url)
root = BeautifulSoup(rs.content, 'html.parser')
english_text = get_mass_result(url)
print(summary1(english_text).replace('<pad>','').replace('</s>',''))

YouTube тестує функцію автоматичної ідентифікації у відео



 YouTube експериментує з новою функцією, яка відображає список продуктів, знайдених у деяких відео. такі продукти відображатимуться між рекомендованими відео для глядачів, які прокручуються під програвачем. мета - допомогти людям знайти більше відео та інформацію про ці продукти на YouTube. 


In [29]:
english_text = get_mass_result(url)
print(summary2(english_text).replace('<pad>','').replace('</s>',''))

YouTube тестує функцію автоматичної ідентифікації у відео



YouTube розпочав новий експеримент, у рамках якого тестується можливість автоматичного визначення списку продуктів у завантажуваних відео. Цей тест було розпочато 22 березня і поширюється на "людей, які переглядають відео в США". Мета - допомогти людям знайти більше відео та інформацію про ці продукти на YouTube.


In [1]:
url = 'https://www.president.gov.ua/news/prishvidshennya-vakcinaciyi-ta-pidtrimku-lyudej-i-biznesu-u-67273'

In [4]:
rs = requests.get(url)
url_text = rs.text
title = url_text.split('<title>')[1].split('</title>')[0]
title

'Пришвидшення вакцинації та підтримку людей і бізнесу у «червоних» зонах обговорили на нараді в Президента — Офіційне інтернет-представництво Президента України'

In [6]:
import html2text

text = url_text.split('<div itemprop="articleBody" class="article_content">')[1].split('</div>')[0]
text = text.replace('(',' ').replace(')',' ')
h = html2text.HTML2Text()
h.ignore_links = True
text = h.handle(text)

answer = []
while text.count('(')!= 0:
    text = text.replace('![','').replace(']','')
    text = text.split('(',1)
    answer = text[0]
    answer+=' '
    answer+=(text[1].split(')',1)[1])
    answer+=' '
    text = answer
    
text = text.replace('\n\n',' ').replace('\n',' ').replace('_',' ')
text

"Пришвидшення вакцинації та підтримку людей і бізнесу у «червоних» зонах обговорили на нараді в Президента     Київ, Вінницька та Івано-Франківська області нині мають найбільшу динаміку зростання кількості хворих на COVID-19. Про це повідомили на селекторній нараді щодо запобігання поширенню коронавірусної хвороби, яка відбулася під головуванням Президента України Володимира Зеленського. Прем'єр-міністр Денис Шмигаль поінформував, що активних хворих в країні станом на сьогодні понад 263 тисячі. «Із 63 тисяч ліжок із підведеним киснем заповнені 39 тисяч. Ліжок інтенсивної терапії зайнято 2 тисячі 652 із 4 тисяч 909», – зазначив голова уряду. Денис Шмигаль додав, що 20 березня було проведено засідання Державної комісії з питань ТЕБ і НС, за підсумками якого до «червоної» зони були віднесені місто Київ та Одеська область. Крім того, сьогодні зранку за дорученням Президента України очільник уряду провів нараду з міністрами щодо розробки механізмів спільної з органами місцевого самоврядуван

In [ ]:
text = [i if len(i)>5 else None for i in text.split('. ')]
while text.count(None) != 0:
    text.remove(None)

english_text_mass = [GoogleTranslator(source='ukrainian', target='english').translate(i) for i in text]
english_text = ''
for i in english_text_mass:
    english_text+=i
    english_text+='. '

In [ ]:
print(summary1(english_text))

In [146]:
print(summary2(english_text))

Президент України Володимир Зеленський привітав українську поетесу Ліну Костенко з днем ​​народження. Глава держави наголосив, що робота Ліни завжди сповнена любові до рідного краю та людської мудрості та правди. "Щиро бажаю, щоб ваша доля повела вас по щасливому життєвому шляху", - додав він.
